# Analysis of cell nuclei morphology

### I - Preprocessing
In this section an example how to generate single nuclei from a segmented and labbeled image is provided.
For this we use a function which takes the following variables as input:
- objects : bounding box for each detected and labelled object in the image
- num_objects: total number of objects detected
- labelled_image: image with labelled objects
- image: original image
- file: filename
- cropsize: size of crop


In [1]:
def format_nuclei(objects,num_objects,labelled_image,image,file,cropsize):
            nuc=ws2[nuclei[num]]
            mask=nuc==0
            crop_1=im[:,:,:,0]
            crop_1=crop_1[nuclei[num]]
            crop_1[mask]=0
            crop_1=np.pad(crop_1,((cropsize,cropsize),(cropsize,cropsize),(cropsize,cropsize)),'constant')
            crop_2=im[:,:,:,1]
            crop_2=crop_2[nuclei[num]]
            crop_2[mask]=0
            crop_2=np.pad(crop_2,((cropsize,cropsize),(cropsize,cropsize),(cropsize,cropsize)),'constant')
            nuc=np.pad(nuc,((cropsize,cropsize),(cropsize,cropsize),(cropsize,cropsize)),'constant')
            cm=center_of_mass(nuc)
            cmi=int(cm[0]),int(cm[1]),int(cm[2])
            binary=nuc[(cmi[0]-cropsize):(cmi[0]+cropsize),(cmi[1]-cropsize):(cmi[1]+cropsize),(cmi[2]-cropsize):(cmi[2]+cropsize)] 
            ch_1=crop_1[(cmi[0]-cropsize):(cmi[0]+cropsize),(cmi[1]-cropsize):(cmi[1]+cropsize),(cmi[2]-cropsize):(cmi[2]+cropsize)]  
            ch_2=crop_2[(cmi[0]-cropsize):(cmi[0]+cropsize),(cmi[1]-cropsize):(cmi[1]+cropsize),(cmi[2]-cropsize):(cmi[2]+cropsize)]  
            new_img=np.stack((img_as_ubyte(binary),img_as_ubyte(ch_1),img_as_ubyte(ch_2)))
            filename=file[:-4]+'_'+str(num+1)+'.tif'
            return io.imsave((path+filename),new_img)     

In [ ]:

        
nuclei=[format_nu(nuclei,num,seg1[0],file) for num in range(0,len(nuclei)) ]   